In [ ]:
!pip install pytorch-warmup
!pip install albumentations

In [ ]:
import os.path as osp

# computation
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import pytorch_warmup as warmup

# data pipeline
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from albumentations import (
    CenterCrop,
    CoarseDropout,
    Compose,
    Cutout,
    HorizontalFlip,
    HueSaturationValue,
    IAAAdditiveGaussianNoise,
    ImageOnlyTransform,
    Normalize,
    OneOf,
    RandomBrightness,
    RandomBrightnessContrast,
    RandomContrast,
    RandomCrop,
    RandomResizedCrop,
    Resize,
    Rotate,
    ShiftScaleRotate,
    Transpose,
    VerticalFlip,
)
from albumentations.pytorch import ToTensorV2

# utils
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torchvision.models as models

import matplotlib.pyplot as plt

In [ ]:
# pytorch GPU 확인

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

# Default CUDA device
cuda = torch.device('cuda:0')

# Dataset

In [ ]:
# Some constants
ROOT = '/kaggle/input/cassava-leaf-disease-classification'
TRAIN_DIR = f'{ROOT}/train_images/'
TRAIN_CSV = f'{ROOT}/train.csv'
TEST_DIR = f'{ROOT}/test_images/'
TEST_CSV = f'{ROOT}/sample_submission.csv'

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, split, transform=None, one_hot=False, label_smoothing=False, label_smoothing_alpha=0.2):
        assert split in ('train', 'val', 'test')
        self.split = split
        self.transform = transform
        if split in ('train', 'val'):
            csv = pd.read_csv(TRAIN_CSV)

            csv_val = pd.DataFrame()
            csv_train = pd.DataFrame()
            for i in range(5):
                csv_label = csv[csv['label']==i]
                csv_label_val = csv_label.sample(n=200, random_state=0)
                csv_label_train = csv_label[~csv_label.index.isin(csv_label_val.index)]
                csv_label_train = csv_label_train.sample(n=2000, random_state=0, replace=True)
                csv_val = pd.concat([csv_val, csv_label_val], axis=0)
                csv_train = pd.concat([csv_train, csv_label_train], axis=0)

            self.df = (csv_train, csv_val)[0 if split == 'train' else 1].reset_index()
        else:
            self.df = pd.read_csv(TEST_CSV)
            
        self.one_hot = one_hot
        self.label_smoothing = label_smoothing
        self.label_smoothing_alpha = label_smoothing_alpha
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, i: int):
        base_dir = TRAIN_DIR if self.split in ('train', 'val') else TEST_DIR
        x = imageio.imread(osp.join(base_dir, self.df['image_id'][i]))
        y = self.df['label'][i] if self.split in ('train', 'val') else -1
        if self.transform:
            x = self.transform(x)
        
        #One-hot encoding
        if self.one_hot:
            y_oh = np.zeros(5)
            y_oh[y] = 1
            y = y_oh
        
        # Label smoothing
        if self.label_smoothing:
            y_ls = y*(1-self.label_smoothing_alpha) + self.label_smoothing_alpha/5
            y = y_ls
        
        y = y.astype('float32')
        return (x, y)

# Implement your data augmentation pipelines

In [ ]:
# TODO: You'll need some heavy augmentations to get a high score. 
#
# See 
#     https://github.com/aleju/imgaug 
#     https://imageio.readthedocs.io/en/stable/
#
# for detailed `imgaug` references.
#
# NOTE: If you choose to normalize the training images, you should normalize the test images as well.
#
INPUT_SIZE = 224

TRANSFORMS = {
    'train': transforms.Compose([
        iaa.Sequential([
            iaa.Resize((255, 255)),
            iaa.CenterCropToFixedSize(INPUT_SIZE,INPUT_SIZE),
            
            # Image augmentation
            iaa.Sometimes(0.25, iaa.convolutional.Sharpen(alpha=(0.0, 0.2))),
            iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 3.0))),
            iaa.Multiply((0.8, 1.2), per_channel=0.2),
#             iaa.Fliplr(0.5),
#             iaa.Flipud(0.5),
#             iaa.Affine(rotate=(-20, 20), mode='symmetric'),
#             iaa.Sometimes(0.25,
#                           iaa.OneOf([iaa.Dropout(p=(0, 0.1)),
#                                      iaa.CoarseDropout(0.1, size_percent=0.5)])),
#             iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)
            
            
        ]).augment_image,
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        iaa.Sequential([
            iaa.Resize((255, 255)),
            iaa.CenterCropToFixedSize(INPUT_SIZE,INPUT_SIZE),
        ]).augment_image,
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        iaa.Sequential([
            iaa.Resize((255, 255)),
            iaa.CenterCropToFixedSize(INPUT_SIZE,INPUT_SIZE),
        ]).augment_image,
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# Build your Cassava leaf classifier

In [ ]:
# TODO: Implement your classifier.

class multiclass_loss_fn(object):
    def __init__(self, loss_weight= []):
        super(multiclass_loss_fn, self).__init__()
        self.loss_weight =  loss_weight
        
        if len(self.loss_weight) > 0:
            loss_weight = np.array(self.loss_weight, dtype='float32')
            loss_weight_norm = loss_weight.sum()
            loss_weight = loss_weight/loss_weight_norm
            loss_weight = loss_weight.reshape(1,-1)
            loss_weight = torch.Tensor(loss_weight).cuda()
            self.loss_weight = loss_weight
        
    def __call__(self, y_hat, y):
        loss_weight = torch.repeat_interleave(self.loss_weight, repeats=y_hat.shape[0], dim=0)
        y_hat = y_hat * loss_weight
        
        res = -nn.LogSoftmax(dim=0)(y_hat)
        res = res * y
        res = torch.sum(res) / y.shape[0]
        return res

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
        
def MixupTraining(batch, alpha, random_state=None):
    img = batch[0]
    lbl = batch[1]
    
    np.random.seed(random_state)
    ratio = np.random.beta(alpha, alpha, size = (len(img),1,1,1)).astype('float32')
    ratioImg = torch.Tensor(ratio)
    ratioImg = torch.repeat_interleave(ratioImg, repeats=img.shape[1], dim=1)
    ratioImg = torch.repeat_interleave(ratioImg, repeats=img.shape[2], dim=2)
    ratioImg = torch.repeat_interleave(ratioImg, repeats=img.shape[3], dim=3)
    
    ratio = ratio.reshape(len(lbl),1)
    ratioLbl = torch.Tensor(ratio)
    ratioLbl = torch.repeat_interleave(ratioLbl, repeats=lbl.shape[1], dim=1)
    
    img2 = torch.flip(img, dims=(0,))
    lbl2 = torch.flip(lbl, dims=(0,))
    mixedImg = torch.add(torch.mul(ratioImg, img), torch.mul(torch.sub(1, ratioImg), img2))
    mixedLbl = torch.add(torch.mul(ratioLbl, lbl), torch.mul(torch.sub(1, ratioLbl), lbl2))
    
    mixedBatch = (mixedImg, mixedLbl)
    return mixedBatch

def make_confMatrix(Y, Y_hat):
    confMatrix = np.zeros(shape=(5,5))
    for y, y_hat in zip(Y_trains, Y_trains_hat):
        confMatrix[int(y), int(y_hat)] += 1
    return confMatrix

class CassavaClassifier(nn.Module):
    def __init__(self):
        super(CassavaClassifier, self).__init__()
        self.resnet = models.wide_resnet50_2(pretrained=True)
        set_parameter_requires_grad(self.resnet, feature_extracting=True)
        self.num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(self.num_ftrs, 200)
        self.b1 = nn.BatchNorm1d(num_features=200)
        self.linear1 = nn.Linear(in_features=200, out_features=5)
        
    def forward(self, x):
        x = self.resnet(x)
        x = self.b1(x)
        x = self.linear1(x)
        return x

# Implement the training loop

In [ ]:
# TODO: Define some hyperparameters here.

In [ ]:
# TODO: Prepare some components for training your network.
#
# See
#
#     Optimizer:    https://pytorch.org/docs/stable/optim.html#how-to-use-an-optimizer
#     LR Scheduler: https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
#     DataLoader:   https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader
#
# for their actual APIs and detailed usages.
trainDataset = CassavaDataset('train', transform=TRANSFORMS['train'], one_hot=True, label_smoothing=False, label_smoothing_alpha=0.3)
valDataset = CassavaDataset('val', transform=TRANSFORMS['val'], one_hot=True, label_smoothing=False, label_smoothing_alpha=0.3)
trainDataLoader = DataLoader(trainDataset, batch_size=128, num_workers=1, shuffle=True)
valDataLoader = DataLoader(valDataset, batch_size=64, shuffle=True)
model = CassavaClassifier().cuda()
loss_fn = multiclass_loss_fn([0.8,1,1,0.7,1])
# loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.01)
# optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0, verbose=True)
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
# warmup_scheduler = warmup.UntunedLinearWarmup(optimizer)
score_fn = accuracy_score

In [ ]:
# TODO: Implement your training loop here.
history = dict()
history['loss'] = dict()
history['acc'] = dict()

history['loss']['train'] = []
history['loss']['val'] = []

history['acc']['train'] = []
history['acc']['val'] = []

loss_weight = [1,1,1,1,1]
for epoch in range(100):
    losses_train = []
    correct_train = 0
    incorrect_train = 0
    
    confMatrix = np.zeros(shape=(5,5))
    loss_fn = multiclass_loss_fn(loss_weight)
    for i,batch in enumerate(tqdm(trainDataLoader)):
        
        # batch = MixupTraining(batch, 0.1, random_state=42)
        
        X_train = batch[0].cuda()
        Y_train = batch[1].reshape(-1,5).cuda()
        # Y_train = batch[1].long().cuda()

        model.train()
        Y_train_hat = model(X_train)
        loss_train = loss_fn(Y_train_hat, Y_train)
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        # warmup_scheduler.dampen()
        
        losses_train.append(loss_train.item())

        Y_train = np.argmax(np.array(Y_train.tolist()),axis=1)
        # Y_train = np.array(Y_train.tolist())
        Y_train_hat = np.argmax(np.array(Y_train_hat.tolist()),axis=1)
        correct_train += (Y_train == Y_train_hat).sum()
        incorrect_train += (Y_train != Y_train_hat).sum()
        
        
        confMatrix += make_confMatrix(Y_train, Y_train_hat)
        
    loss_weight = [1/confMatrix[i,i] for i in range(5)]


            
    losses_val = []
    correct_val = 0
    incorrect_val = 0
    for j,batch in enumerate(tqdm(valDataLoader)):
        X_val = batch[0].cuda()
        Y_val = batch[1].reshape(-1,5).cuda()
        # Y_val = batch[1].long().cuda()

        model.eval()
        Y_val_hat = model(X_val)
        loss_val = loss_fn(Y_val_hat, Y_val)
        
        losses_val.append(loss_val.item())

        Y_val = np.argmax(np.array(Y_val.tolist()),axis=1)
        # Y_val = np.array(Y_val.tolist())
        Y_val_hat = np.argmax(np.array(Y_val_hat.tolist()),axis=1)
        correct_val += (Y_val == Y_val_hat).sum()
        incorrect_val += (Y_val != Y_val_hat).sum()
        
    loss_train_epoch = sum(losses_train)/len(losses_train)
    loss_val_epoch = sum(losses_val)/len(losses_val)
    acc_train_epoch = correct_train / (correct_train + incorrect_train)
    acc_val_epoch = correct_val / (correct_val + incorrect_val)
    
    history['loss']['train'].append( loss_train_epoch )
    history['loss']['val'].append( loss_val_epoch )
    history['acc']['train'].append( acc_train_epoch )
    history['acc']['val'].append( acc_val_epoch )
        
    print('epoch %d' %(epoch+1))
    print('train >> loss: %.8f \t acc: %.8f' % (loss_train_epoch,acc_train_epoch))
    print(' val  >> loss: %.8f \t acc: %.8f' % (loss_val_epoch,acc_val_epoch))

    if epoch > 1:
        if acc_val_epoch >= max(history['acc']['val']):
            print('best model is updated.')
            torch.save(model, 'best_model.pt')

        if max(history['acc']['val'][-5:]) - min(history['acc']['val'][-5:]) < 0.0001:
            break


    scheduler.step(acc_val_epoch)
    # scheduler.step()

    plt.figure(figsize=(10,5))
    plt.subplot(2,1,1)
    plt.plot(history['loss']['train'], color='black')
    plt.plot(history['loss']['val'], color='r')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.subplot(2,1,2)
    plt.plot(history['acc']['train'], color='black', label='train')
    plt.plot(history['acc']['val'], color='r', label='test')
    plt.xlabel('epoch')
    plt.ylabel('acc')
    plt.legend()
    plt.show()


print('Finished Training')

In [ ]:
import seaborn as sns

Y_trains = np.array([])
Y_trains_hat = np.array([])
for i,batch in enumerate(tqdm(trainDataLoader)):
        
        # batch = MixupTraining(batch, 0.1, random_state=42)
        
        X_train = batch[0].cuda()
        Y_train = batch[1].reshape(-1,5).cuda()
        
        model.eval()
        Y_train_hat = model(X_train)
        
        Y_train = np.argmax(np.array(Y_train.tolist()),axis=1)
        Y_train_hat = np.argmax(np.array(Y_train_hat.tolist()),axis=1)
        
        Y_trains = np.concatenate((Y_trains, Y_train), axis=0)
        Y_trains_hat = np.concatenate((Y_trains_hat, Y_train_hat), axis=0)

        
        
confMatrix = make_confMatrix(Y_train, Y_train_hat)
    
print(confMatrix.astype(int))

sns.heatmap(confMatrix, cmap='RdBu').invert_yaxis()
plt.ylabel('Y')
plt.xlabel('Y_hat')
plt.show()

In [ ]:
import seaborn as sns


Y_trains = np.array([])
Y_trains_hat = np.array([])
for i,batch in enumerate(tqdm(valDataLoader)):
        
        # batch = MixupTraining(batch, 0.1, random_state=42)
        
        X_train = batch[0].cuda()
        Y_train = batch[1].reshape(-1,5).cuda()
        
        model.eval()
        Y_train_hat = model(X_train)
        
        Y_train = np.argmax(np.array(Y_train.tolist()),axis=1)
        Y_train_hat = np.argmax(np.array(Y_train_hat.tolist()),axis=1)
        
        Y_trains = np.concatenate((Y_trains, Y_train), axis=0)
        Y_trains_hat = np.concatenate((Y_trains_hat, Y_train_hat), axis=0)

        
        
confMatrix = make_confMatrix(Y_train, Y_train_hat)


sns.heatmap(confMatrix, cmap='RdBu').invert_yaxis()
plt.ylabel('Y')
plt.xlabel('Y_hat')
plt.show()

In [ ]:
# Save your trained model. Note that it is recommended to save the 'best' checkpoint, rather than just saving the 'last' checkpoint.
torch.save(best_mobel, 'model.pt')

# Verify that your trained model can be loaded

In [ ]:
model = torch.load('/kaggle/working/model.pt')
model.eval()
test_dataset = CassavaDataset('test', transform=TRANSFORMS['test'])
test_loader = DataLoader(test_dataset, batch_size=32)
test_csv = pd.read_csv(TEST_CSV)

y_hats = []
for x, _ in test_loader:
    y_hat = model(x.cuda())
    y_hat = torch.argmax(y_hat,dim=1)
    y_hats.extend(y_hat.cpu().detach().numpy().tolist())
    

test_csv['label'] = y_hats
test_csv[['image_id','label']].to_csv("submission.csv", index=False)
test_csv.head()